# Example 11: Random Environments and SolverENV

This example illustrates how to model a queueing system operating in a random environment, where system parameters (e.g., service rates) change according to an underlying environmental process.

**Scenario**: A server that alternates between "Fast" and "Slow" modes.
- In Fast mode, service rate is 4.0
- In Slow mode, service rate is 1.0
- The environment switches from Fast→Slow at rate 0.5 and Slow→Fast at rate 1.0

In [ ]:
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.env.*
import jline.solvers.fluid.*
import jline.solvers.mva.*

GlobalConstants.setVerbose(VerboseLevel.STD)

## Block 1: Create Base Network Model

Define a closed queueing network with a delay node (think time) and a queue (server).

In [ ]:
// Create the base network model
val baseModel = Network("BaseModel")
val delay = Delay(baseModel, "ThinkTime")
val queue = Queue(baseModel, "Fast/Slow Server", SchedStrategy.FCFS)

// Define a closed class with 5 jobs
val N = 5
val jobclass = ClosedClass(baseModel, "Jobs", N, delay, 0)

// Set service rates (placeholder for base model)
delay.setService(jobclass, Exp(1.0))   // Think time = 1.0
queue.setService(jobclass, Exp(2.0))   // Placeholder service rate

// Connect nodes in a cycle
val P = baseModel.initRoutingMatrix()
P.addRoute(jobclass, delay, queue, 1.0)
P.addRoute(jobclass, queue, delay, 1.0)
baseModel.link(P)

## Block 2: Create the Random Environment

Define two stages (Fast and Slow modes) with different service rates and transition rates.

In [ ]:
// Create environment with two stages
val env = Environment("ServerModes")

// Stage 0: Fast mode (service rate = 4.0)
val fastModel = baseModel.copy()
val fastQueue = fastModel.getNodeByName("Fast/Slow Server")
fastQueue.setService(fastModel.classes[0], Exp(4.0))
env.addStage(0, "Fast", "operational", fastModel)

// Stage 1: Slow mode (service rate = 1.0)
val slowModel = baseModel.copy()
val slowQueue = slowModel.getNodeByName("Fast/Slow Server")
slowQueue.setService(slowModel.classes[0], Exp(1.0))
env.addStage(1, "Slow", "degraded", slowModel)

// Define transitions between stages
// Fast -> Slow at rate 0.5 (mean time in Fast mode = 2.0)
env.addTransition(0, 1, Exp(0.5))
// Slow -> Fast at rate 1.0 (mean time in Slow mode = 1.0)
env.addTransition(1, 0, Exp(1.0))

## Block 3: Inspect Environment Structure

In [ ]:
// Display environment structure
println("Environment stages:")
val stageTable = env.getStageTable()
stageTable.print()

## Block 4: Solve Using SolverENV

SolverENV computes environment-averaged metrics using a solver factory that creates solvers for each stage.

In [ ]:
// Create solver factory that builds Fluid solver for each stage
val solverFactory = { m: Network -> FLD(m) }

// Create and run the ENV solver
val envSolver = ENV(env, solverFactory)
val qAvg = envSolver.getAvg()

// Display average results weighted by environment probabilities
println("\n--- Environment-Averaged Results ---")
val envAvgTable = envSolver.getAvgTable()
envAvgTable.print()

## Block 5: Compare with Individual Stage Analysis

Analyze each stage network in isolation using steady-state solvers like MVA.

In [ ]:
// Compare with individual stage analysis using MVA
println("\n--- Individual Stage Analysis (MVA) ---")
val ensemble = env.getEnsemble()
for ((idx, stageModel) in ensemble.withIndex()) {
    println("\nStage $idx:")
    val mvaSolver = MVA(stageModel)
    val stageTable = mvaSolver.getAvgTable()
    stageTable.print()
}

## Key Observations

1. **Environment-averaged results**: SolverENV accounts for the probability of being in each stage over time
2. **Stage probabilities**: The environment has a steady-state distribution that weights the metrics
3. **Individual vs. averaged**: Each stage has different performance (e.g., different response times in Fast vs. Slow modes)
4. **Modulation effect**: The random environment introduces variability in system performance